In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
from tqdm.autonotebook import tqdm, trange
import itertools
from matplotlib import pyplot as plt

import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
import sys
sys.path.append('../structural')

import evaluation
import importlib
import getInvalid
importlib.reload(evaluation)
importlib.reload(getInvalid)

C:\Users\Lyle\AppData\Local\Temp\ipykernel_5176\387944769.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


<module 'getInvalid' from 'C:\\Users\\Lyle\\Documents\\DeCoDE\\DeCoDE-Bike-Project-Git\\Structural Analysis\\../structural\\getInvalid.py'>

In [2]:
x_scaled, y, _, xscaler = load_data.load_framed_dataset("r", onehot = True, scaled = True, augmented = True)

y_scaler = StandardScaler()
y_scaler.fit(y.values)
y_scaled = y_scaler.transform(y.values)

# #Scale y values and subtract from 1 to turn into a maximization problem for MO-PaDGAN
# y_MMscaler = MinMaxScaler()
# y_MMscaler.fit(y)
# y_MMscaled = y_MMscaler.transform(y)
# y_MMscaled=1-y_scaled

NameError: name 'load_data' is not defined

In [5]:
def tf_inv_scale(y_scaled):
    scale=tf.cast(tf.convert_to_tensor(y_scaler.scale_), "float32")
    mean=tf.cast(tf.convert_to_tensor(y_scaler.mean_), "float32")
    return tf.cast(y_scaled*scale+mean, "float32")
print(tf_inv_scale(y_scaled))

tf.Tensor(
[[1.79999992e-02 3.61000001e-02 2.15999987e-02 ... 2.86532950e+00
  9.61538434e-01 4.73000002e+00]
 [1.31299999e-02 1.59499999e-02 1.26499999e-02 ... 1.61290324e+00
  1.29920745e+00 1.36000013e+00]
 [5.33999968e-03 7.61999935e-03 5.96000068e-03 ... 1.34048259e+00
  8.26446295e-01 2.76000023e+00]
 ...
 [4.32199985e-03 1.15600005e-02 4.08899970e-03 ... 1.47318804e+00
  2.89017344e+00 5.68000031e+00]
 [9.61299986e-04 3.17500159e-03 9.21299681e-04 ... 4.06669378e-01
  1.07492208e+00 7.59999990e+00]
 [4.45099995e-02 9.08399969e-02 5.23699969e-02 ... 6.84931564e+00
  2.71149683e+00 3.44000006e+00]], shape=(4046, 10), dtype=float32)


In [6]:
def evaluate_DTAI(y, DTAI_EPS=1e-7):
    ref = evaluation.percentileSelect(y_orig, 0.25)
    p_=[1, 1, 1, 1, 2, 1, 1, 4, 4, 3] #alpha
    a_=[4, 4, 4, 4, 4, 4, 4, 4, 4, 2] #beta
    p_=tf.cast(p_, "float32")
    a_=tf.cast(a_, "float32")

    #y values must be greater than 0
    y=tf.math.maximum(y, DTAI_EPS)
    x=tf.divide(ref,y)
    case1 = tf.multiply(p_,x)-p_
    p_over_a=tf.divide(p_,a_)
    exponential=tf.exp(tf.multiply(a_, (1-x)))
    case2=tf.multiply(p_over_a, (1-exponential))
    casemask = tf.greater(x, 1)
    casemask = tf.cast(casemask, "float32")
    scores=tf.multiply(case2, casemask) + tf.multiply( case1, (1 - casemask))
    scores=tf.math.reduce_sum(scores, axis=1)         
    smax=tf.math.reduce_sum(p_/a_)
    smin=-tf.math.reduce_sum(p_)

    scores=(scores-smin)/(smax-smin)
    return scores

def evaluate_DTAI_2(y, DTAI_EPS=1e-7):
    ref = evaluation.percentileSelect(y_orig, 0.25)
    p_=[1, 1, 1, 1, 2, 1, 1, 4, 4, 3] #alpha
    a_=[4, 4, 4, 4, 4, 4, 4, 4, 4, 2] #beta
    p_=tf.cast(p_, "float32")
    a_=tf.cast(a_, "float32")

    #y values must be greater than 0
    y=tf.math.maximum(y, DTAI_EPS)
    x=tf.divide(ref,y)
    case1 = tf.multiply(p_,x)-p_
    p_over_a=tf.divide(p_,a_)
    exponential=tf.exp(tf.multiply(a_, (1-x)))
    case2=tf.multiply(p_over_a, (1-exponential))
    casemask = tf.greater(x, 1)
    casemask = tf.cast(casemask, "float32")
    scores=tf.multiply(case2, casemask) + tf.multiply( case1, (1 - casemask))
        
    mask=tf.random.uniform(tf.shape(y), minval=0, maxval=1)
    masksums=tf.reduce_sum(mask, axis=1)
    masksums=tf.reshape(masksums, (-1, 1))
    mask=tf.divide(mask, masksums)
    
    
    scores=tf.multiply(scores, mask)
    
    smax = tf.multiply(tf.divide(p_, a_), mask)
    smax = tf.math.reduce_sum(smax, axis=1) 
    smin = -tf.multiply(p_, mask)
    smin = tf.math.reduce_sum(smin, axis=1) 
    scores=tf.math.reduce_sum(scores, axis=1) 
    scores=tf.divide(tf.subtract(scores,smin), tf.subtract(smax,smin))

    return scores

#MO-PaDGAN scoring function using random weighting of objectives
def evaluate_MO(y):
    ref = evaluation.percentileSelect(y_orig, 0.5)
    ref = tf.constant(ref, dtype="float32")
    y=tf.math.maximum(y, 0.0)
    y=tf.divide(ref,(ref+y))
#     slicelist=[]
    mask=tf.random.uniform(tf.shape(y), minval=0, maxval=1)
    masksums=tf.reduce_sum(mask, axis=1)
    masksums=tf.reshape(masksums, (-1, 1))
    mask=tf.divide(mask, masksums)
    scores=tf.multiply(y, mask)
    scores=tf.reduce_sum(scores, axis=1)
    return scores

#Test Evaluation Functions
# print(evaluate_DTAI(y))
# print(evaluate_MO(y))


In [7]:
y=tf.cast(tf.convert_to_tensor(y), "float32")
print(evaluate_DTAI(y))
print(evaluate_DTAI_2(y))
print(evaluate_MO(y))

tf.Tensor([0.4207287  0.59287995 0.72590697 ... 0.48619732 0.7558108  0.26138914], shape=(4046,), dtype=float32)
tf.Tensor([0.4020614  0.59579533 0.73877406 ... 0.47677636 0.7461123  0.3152979 ], shape=(4046,), dtype=float32)
tf.Tensor([0.46230447 0.5830029  0.66625154 ... 0.5165515  0.67345935 0.28364068], shape=(4046,), dtype=float32)


In [8]:
class discriminator(keras.Model):
    def __init__(self):
        super(discriminator, self).__init__()
        
        self.Dense1 = keras.layers.Dense(128)
        self.LReLU1 = keras.layers.LeakyReLU(alpha = 0.2)
        
        self.Dense2 = keras.layers.Dense(128)
        self.LReLU2 = keras.layers.LeakyReLU(alpha = 0.2)
        
        self.Dense3 = keras.layers.Dense(1)
    
    def call(self, inputs):
        
        x = self.Dense1(inputs)
        x = self.LReLU1(x)
        
        x = self.Dense2(x)
        x = self.LReLU2(x)
        
        x = self.Dense3(x)
        
        return x

In [9]:
class generator(keras.Model):
    def __init__(self, data_dim):
        super(generator, self).__init__()
        
        self.Dense1 = keras.layers.Dense(128)
        self.LReLU1 = keras.layers.LeakyReLU(alpha = 0.2)
        
        self.Dense2 = keras.layers.Dense(128)
        self.LReLU2 = keras.layers.LeakyReLU(alpha = 0.2)
        
        self.Dense3 = keras.layers.Dense(128)
        self.LReLU3 = keras.layers.LeakyReLU(alpha = 0.2)
        
        self.Dense4 = keras.layers.Dense(data_dim)
#         self.out_activation = keras.layers.Activation(keras.activations.tanh)
        
    def call(self, inputs):
        
        x = self.Dense1(inputs)
        x = self.LReLU1(x)
        
        x = self.Dense2(x)
        x = self.LReLU2(x)
        
        x = self.Dense3(x)
        x = self.LReLU3(x)
        
        x = self.Dense4(x)
        x = self.LReLU2(x)
        
        return x

In [10]:
class PadGAN(object):
    def __init__(self, noise_dim = 2, data_dim = 2, lambda0 = 2.0, lambda1 = 0.5):

        self.noise_dim = noise_dim
        self.data_dim = data_dim
        self.lambda0 = lambda0
        self.lambda1 = lambda1
        self.EPSILON = 1e-7

        self.discriminator = discriminator()

        self.generator = generator(data_dim)
    def compute_diversity_loss(self, x, y):
        
        r = tf.reduce_sum(tf.math.square(x), axis=1, keepdims=True)
        D = r - 2 * tf.matmul(x, tf.transpose(x)) + tf.transpose(r)
        S = tf.exp(-0.5 * tf.math.square(D))
        y = tf.squeeze(y)
        
        if self.lambda0 == 'inf':

            eig_val, _ = tf.linalg.eigh(S)
            loss = -10 * tf.reduce_mean(y)

            Q = None
            L = None

        elif self.lambda0 == 'naive':

            eig_val, _ = tf.linalg.eigh(S)
            loss = -tf.reduce_mean(tf.math.log(tf.math.maximum(eig_val, self.EPSILON))) - 10 * tf.reduce_mean(y)

            Q = None
            L = None

        else:
            Q = tf.tensordot(tf.expand_dims(y, 1), tf.expand_dims(y, 0), 1)
            if self.lambda0 == 0.:
                L = S
            else:
                L = S * tf.math.pow(Q, self.lambda0)

            eig_val, _ = tf.linalg.eigh(L)
            loss = -tf.reduce_mean(tf.math.log(tf.math.maximum(eig_val, self.EPSILON)))

        return loss, D, S, Q, L, y
    
    @tf.function
    def train_step(self, X_batch, equation, d_optimizer, g_optimizer):
        binary_cross_entropy_loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
        batch_size = X_batch.shape[0]
        
        with tf.GradientTape() as tape:
            d_real = self.discriminator(X_batch)
            d_loss_real = binary_cross_entropy_loss_fn(tf.ones_like(d_real),d_real)
        variables = self.discriminator.trainable_weights
        gradients = tape.gradient(d_loss_real, variables)
        d_optimizer.apply_gradients(zip(gradients, variables))
        z = tf.random.normal(stddev=0.5,shape=(batch_size, self.noise_dim))
        X_fake = self.generator(z)
        with tf.GradientTape() as tape:
            d_fake = self.discriminator(X_fake)
            d_loss_fake = binary_cross_entropy_loss_fn(tf.zeros_like(d_fake),d_fake)
        variables = self.discriminator.trainable_weights
        gradients = tape.gradient(d_loss_fake, variables)
        d_optimizer.apply_gradients(zip(gradients, variables))
        
        z = tf.random.normal(stddev=0.5,shape=(batch_size, self.noise_dim))
        
        with tf.GradientTape() as tape:
            x_fake_train = self.generator(z)
            d_fake = self.discriminator(x_fake_train)
            g_loss = binary_cross_entropy_loss_fn(tf.ones_like(d_fake),d_fake)

            y = tf.expand_dims(equation(x_fake_train),-1)
            dpp_loss = self.compute_diversity_loss(x_fake_train, y)[0]
            g_dpp_loss = g_loss + self.lambda1 * dpp_loss
        
        variables = self.generator.trainable_weights
        gradients = tape.gradient(g_dpp_loss, variables)
        g_optimizer.apply_gradients(zip(gradients, variables))
        
        mean_y = tf.reduce_mean(y)
        
        return d_loss_real, d_loss_fake, g_loss, dpp_loss, mean_y
    
    def train(self, X, equation, steps=10000, batch_size=8, disc_lr=1e-4, gen_lr=1e-4):
        g_optimizer = keras.optimizers.Adam(learning_rate = gen_lr, beta_1=0.5)
        d_optimizer = keras.optimizers.Adam(learning_rate = disc_lr, beta_1=0.5)
        
        steps_range = trange(steps, desc='GAN Training:', leave=True, ascii ="         =")
        
        for step in steps_range:
            ind = np.random.choice(X.shape[0], size=batch_size, replace=False)
            X_batch = tf.cast(X[ind],tf.float32)
            d_loss_real, d_loss_fake, g_loss, dpp_loss, mean_y = self.train_step(X_batch, equation, d_optimizer, g_optimizer)
            
            steps_range.set_postfix_str(
                    "[D] R = %+.7f, F = %+.7f [G] F = %+.7f, dpp = %+.7f,  y = %+.7f" % (d_loss_real, d_loss_fake,
                                                                                         g_loss, dpp_loss, mean_y))

In [11]:
#Load Regression Model
regressor=tf.keras.models.load_model("./Trained Models/BestNN")
regressor.trainable=False

scaling=True #IMPORTANT! Switch this line if your regressor does not use scaled y vals

#Load Classifier
clf=tf.keras.models.load_model("./Trained Models/BestNN_Classifier_final")
clf.trainable=False

# print(y_scaler.inverse_transform(regressor(x_scaled)))
# print(y)
if scaling==True:
    print(sklearn.metrics.r2_score(y, y_scaler.inverse_transform(regressor(x_scaled))))
else:
    print(sklearn.metrics.r2_score(y, regressor(x_scaled)))

0.6974228560294983


In [12]:
def evaluation_function(x_scaled, dtai, val, other=0):
    if other==0:
        y_pred = regressor(x_scaled)
    else:
        x_scaled=pd.DataFrame(x_scaled.numpy(), columns=x_orig.columns)
        y_pred = gluon_reg.predict(x_scaled)
    if scaling==True:
        
        if other==0:
            y_pred = tf_inv_scale(y_pred)
        else:
            y_pred = tf.cast(y_scaler.inverse_transform(y_pred), "float32")
    if dtai==1:
        perfscore = evaluate_DTAI(y_pred)
#         print(perfscore)
    elif dtai==2:
        perfscore = evaluate_DTAI_2(y_pred)
#         print(perfscore)
    else:
        perfscore = evaluate_MO(y_pred)
#         print(perfscore)
#         perfscore = tf.math.reciprocal(perfscore)
    if val==True:
        if other==0:
            classes=clf(x_scaled)
            classes = tf.squeeze(classes)
#         print(classes)
            classes = tf.ones(tf.shape(classes))-classes
            return tf.multiply(perfscore,classes)
        else:
            classes=gluon_clf.predict_proba(x_scaled)
            classes=classes[0]
            return perfscore*classes

        
    else:
        return perfscore
# def evaluation_function_MO(x_scaled):
#     y_pred=regressor(x_scaled)
#     if scaling==True:
#         y_pred = tf_inv_scale(y_pred)
#     return evaluate_MO(y_pred)
# def evaluation_function_validity(x_scaled):
#     y_pred=regressor(x_scaled)
#     if scaling==True:
#         y_pred = tf_inv_scale(y_pred)
    
#     res=tf.multiply(evaluate_DTAI(y_pred),classes)
#     return res


In [13]:
def eval_wrapper(x_scaled):
    return evaluation_function(x_scaled, dtai=1, val=True)
noise_dim = 4
model = PadGAN(lambda0=5, lambda1=0.5, noise_dim = noise_dim, data_dim = len(x_scaled[0,:]))
model.train(x_scaled, eval_wrapper, steps=50000)

GAN Training::   0%|          | 0/50000 [00:00<?, ?it/s]

In [18]:
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.factory import get_problem
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter
from pymoo.core.problem import ElementwiseProblem

df=pd.DataFrame(x_scaled)
lowers=-np.ones(noise_dim)
uppers=np.ones(noise_dim)

lowers_gen=df.quantile(0.02).values
uppers_gen=df.quantile(0.98).values

print(lowers_gen)
print(uppers_gen)

[-1.20257688 -0.47350371 -0.5398277  -0.82138228 -0.79878449 -1.39448295
 -2.48069215 -2.31601858 -0.92954969 -2.0503459  -1.35424495 -0.97230762
 -1.8711344  -1.10830508 -1.5979023  -1.18584824 -1.29003608 -2.81899736
 -1.76841494 -0.93773878 -1.5620935  -2.32828212 -1.43204445 -2.55574529
 -0.20949289 -1.49849224 -2.17067456 -3.60957837 -0.10459606 -1.45251131
 -1.52737117 -1.03794992 -1.03219342 -1.03018236 -1.02570751 -1.02358773
 -1.02991138 -1.03380356 -2.24622228]
[ 0.83154768  2.11191583  1.85244286  1.21745992  1.2519021   1.74239779
  0.86129463  2.80671732  3.48917603  1.48371243  2.49777389  2.82385921
  2.06960177  2.85538683  2.9580915   2.36601567  2.39090037  0.08019734
  2.36369944  3.15631459  2.18596578  1.56955135  2.30581284  1.60726002
  4.05312634  1.63475978  0.13442628  0.315934   -0.10459606  1.55862069
  2.79275823  2.44777989  2.44339843  2.43646193  2.42196321  2.445941
  2.43164957  2.40445759  1.84305301]


In [19]:
tens = tf.cast(tf.convert_to_tensor(lowers), "float32")
tens = tf.expand_dims(tens, axis=0)
x=model.generator(tens)
print(gen)

NameError: name 'gen' is not defined

In [45]:
regressor=tf.keras.models.load_model("./Trained Models/BestNN")
regressor.trainable=False
classifier=tf.keras.models.load_model("./Trained Models/BestNN_Classifier_final")
classifier.trainable=False
class build_problem(ElementwiseProblem):
    def __init__(self, lowers, uppers):
        super().__init__(n_var=4,
                         n_obj=10,
                         n_constr=3,
                         xl=lowers,
                         xu=uppers)
    def _evaluate(self, x, out, *args, **kwargs):
        tens = tf.cast(tf.convert_to_tensor(x), "float32")
        tens = tf.expand_dims(tens, axis=0)
        x_gen = model.generator(tens)
        invalidprob = classifier.predict(x_gen)
        out["F"] = regressor.predict(x_gen)
        x_gen = x_gen.numpy()
        out["G"] = [invalidprob[0][0]-0.1, any(i<0 for i in x_gen[0]-lowers_gen[0]), any(i<0 for i in uppers_gen[0]-x_gen[0])]
        
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.factory import get_sampling, get_crossover, get_mutation
from pymoo.factory import get_termination

termination = get_termination("n_gen", 200)
algorithm = NSGA2(
    pop_size=250,
    n_offsprings=10,
    sampling=get_sampling("real_random"),
    crossover=get_crossover("real_sbx", prob=0.9, eta=15),
    mutation=get_mutation("real_pm", eta=20),
    eliminate_duplicates=True
)

In [46]:
from pymoo.optimize import minimize
problem = build_problem(lowers, uppers)

print(problem)
res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               save_history=True,
               verbose=True)

# name: build_problem
# n_var: 4
# n_obj: 10
# n_constr: 3

1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 24ms/step
n_gen |  n_eval |   cv (min)   |   cv (avg)   |  n_nds  |     eps      |  indicator  
    1 |     250 |  1.000000000 |  1.820090157 |       1 |            - |            -
1/1 [==============================] - 0s 24ms/step
    2 |     260 |  1.000000000 |  1.808000000 |       1 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
    3 |     270 |  1.000000000 |  1.792000000 |       1 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
    4 |     280 |  1.000000000 |  1.780000000 |       1 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
    5 |     290 |  1.000000000 |  1.760000000 |       1 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
    6 |     300 |  1.000000000 |  1.756000000 |       1 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 24ms/step
    7 |     310 |  1.000000000 |  1.752000000 |       1 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
    8 |     320 |  1.000000000 |  1.736000000 |       1 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
    9 |     330 |  1.000000000 |  1.728000000 |       1 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
   10 |     340 |  1.000000000 |  1.720000000 |       1 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
   11 |     350 |  1.000000000 |  1.708000000 |       1 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
   12 |     360 |  1.000000000 |  1.688000000 |       1 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
   13 |     370 |  1.000000000 |  1.672000000 |       1 |  0.00000E+00 |            f
1/1 [=============================

1/1 [==============================] - 0s 24ms/step
   14 |     380 |  1.000000000 |  1.672000000 |       1 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
   15 |     390 |  1.000000000 |  1.652000000 |       1 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
   16 |     400 |  1.000000000 |  1.616000000 |       1 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
   17 |     410 |  0.00000E+00 |  1.584000000 |       1 |  0.411150388 |        ideal
1/1 [==============================] - 0s 24ms/step
   18 |     420 |  0.00000E+00 |  1.568000000 |       1 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
   19 |     430 |  0.00000E+00 |  1.544000000 |       1 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
   20 |     440 |  0.00000E+00 |  1.524000000 |       1 |  0.00000E+00 |            f
1/1 [=============================

1/1 [==============================] - 0s 24ms/step
   29 |     530 |  0.00000E+00 |  1.340000000 |       1 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
   30 |     540 |  0.00000E+00 |  1.320000000 |       1 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
   31 |     550 |  0.00000E+00 |  1.292000000 |       1 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
   32 |     560 |  0.00000E+00 |  1.272000000 |       1 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
   33 |     570 |  0.00000E+00 |  1.236000000 |       1 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
   34 |     580 |  0.00000E+00 |  1.212000000 |       1 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
   35 |     590 |  0.00000E+00 |  1.200000000 |       1 |  0.00000E+00 |            f
1/1 [=============================

...

1/1 [==============================] - 0s 24ms/step
  160 |    1840 |  0.00000E+00 |  0.980000000 |       5 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
  161 |    1850 |  0.00000E+00 |  0.980000000 |       5 |  0.00000E+00 |            f
1/1 [==============================] - 0s 25ms/step
  162 |    1860 |  0.00000E+00 |  0.976000000 |       5 |  0.182358016 |        ideal
1/1 [==============================] - 0s 24ms/step
  163 |    1870 |  0.00000E+00 |  0.976000000 |       5 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
  164 |    1880 |  0.00000E+00 |  0.976000000 |       5 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
  165 |    1890 |  0.00000E+00 |  0.972000000 |       6 |  0.223928234 |        ideal
1/1 [==============================] - 0s 32ms/step
  166 |    1900 |  0.00000E+00 |  0.972000000 |       6 |  0.00000E+00 |            f
1/1 [=============================

1/1 [==============================] - 0s 24ms/step
  167 |    1910 |  0.00000E+00 |  0.972000000 |       6 |  0.00000E+00 |            f
1/1 [==============================] - 0s 32ms/step
  168 |    1920 |  0.00000E+00 |  0.972000000 |       6 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
  169 |    1930 |  0.00000E+00 |  0.972000000 |       6 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
  170 |    1940 |  0.00000E+00 |  0.972000000 |       6 |  0.00000E+00 |            f
1/1 [==============================] - 0s 32ms/step
  171 |    1950 |  0.00000E+00 |  0.972000000 |       6 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
  172 |    1960 |  0.00000E+00 |  0.968000000 |       7 |  0.061150658 |            f
1/1 [==============================] - 0s 24ms/step
  173 |    1970 |  0.00000E+00 |  0.968000000 |       7 |  0.00000E+00 |            f
1/1 [=============================

1/1 [==============================] - 0s 32ms/step
  174 |    1980 |  0.00000E+00 |  0.968000000 |       7 |  0.00000E+00 |            f
1/1 [==============================] - 0s 32ms/step
  175 |    1990 |  0.00000E+00 |  0.968000000 |       7 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
  176 |    2000 |  0.00000E+00 |  0.968000000 |       7 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
  177 |    2010 |  0.00000E+00 |  0.968000000 |       7 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
  178 |    2020 |  0.00000E+00 |  0.964000000 |       8 |  0.034325992 |            f
1/1 [==============================] - 0s 32ms/step
  179 |    2030 |  0.00000E+00 |  0.964000000 |       8 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
  180 |    2040 |  0.00000E+00 |  0.964000000 |       8 |  0.00000E+00 |            f
1/1 [=============================

1/1 [==============================] - 0s 32ms/step
  189 |    2130 |  0.00000E+00 |  0.948000000 |      12 |  0.080589839 |        ideal
1/1 [==============================] - 0s 24ms/step
  190 |    2140 |  0.00000E+00 |  0.948000000 |      12 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
  191 |    2150 |  0.00000E+00 |  0.948000000 |      12 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
  192 |    2160 |  0.00000E+00 |  0.948000000 |      12 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
  193 |    2170 |  0.00000E+00 |  0.948000000 |      12 |  0.00000E+00 |            f
1/1 [==============================] - 0s 32ms/step
  194 |    2180 |  0.00000E+00 |  0.948000000 |      12 |  0.00000E+00 |            f
1/1 [==============================] - 0s 32ms/step
  195 |    2190 |  0.00000E+00 |  0.948000000 |      12 |  0.00000E+00 |            f
1/1 [=============================

1/1 [==============================] - 0s 24ms/step
  196 |    2200 |  0.00000E+00 |  0.948000000 |      12 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
  197 |    2210 |  0.00000E+00 |  0.948000000 |      12 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
  198 |    2220 |  0.00000E+00 |  0.948000000 |      12 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
  199 |    2230 |  0.00000E+00 |  0.948000000 |      12 |  0.00000E+00 |            f
1/1 [==============================] - 0s 24ms/step
  200 |    2240 |  0.00000E+00 |  0.948000000 |      12 |  0.00000E+00 |            f


In [670]:
z = tf.random.normal((250, noise_dim), stddev = 0.5, dtype=tf.float32)
x_fake = model.generator(z, training=False)
print(x_fake)

tf.Tensor(
[[ 0.55271244 -0.48963436 -0.07436447 ...  0.24791767  0.6968918
  -0.09750409]
 [ 0.42612892 -0.5728347  -0.02084429 ... -0.45939493 -0.18026891
  -0.3233646 ]
 [ 1.053212   -1.2630205  -0.39934927 ... -0.22152133  0.36345294
  -0.37663162]
 ...
 [-0.18477339 -0.30284196  0.9213487  ... -0.4719599   1.7811157
  -0.32667494]
 [ 1.7832397  -0.845757   -0.13212597 ... -0.87144035  2.6086493
  -1.1936376 ]
 [ 1.3579383  -1.4090345  -0.307821   ... -0.17704399  1.9913054
  -0.4310995 ]], shape=(250, 39), dtype=float32)


In [671]:
result=res.pop.get("X")
tens = tf.cast(tf.convert_to_tensor(result), "float32")
x_fake=model.generator(tens)
print(x_fake)

tf.Tensor(
[[-2.3948984  -0.49091205  2.1194282  ... -2.8773139   4.981057
  -1.2658509 ]
 [ 0.89574766 -0.8200351  -0.2011746  ...  0.93213737  1.0526801
  -0.04864093]
 [ 2.1708267  -2.0485387  -0.43724424 ... -0.5315735   1.9585347
  -0.95184773]
 ...
 [ 1.0082314  -0.8725004  -0.12507312 ... -0.00499695  1.3828936
  -0.11771935]
 [-1.2218394  -0.3465749   1.8071579  ... -1.502972    1.6592438
  -0.49493286]
 [-1.3453362  -0.36303934  1.7190019  ... -1.7600545   2.443134
  -0.6651637 ]], shape=(250, 39), dtype=float32)


In [672]:
# print(x_scaled)
# print(evaluation_function(x_fake))
# print(evaluation_function(x_scaled))

x_fake_descaled = xscaler.inverse_transform(x_fake)
df = pd.DataFrame(x_fake_descaled, columns=x_orig.columns)
mats=np.full(len(df.index), "Material_Placeholder")
for i in range(len(df.index)):
    if df.loc[i, "Material=Steel"] > df.loc[i, "Material=Aluminum"]:
        if df.loc[i, "Material=Steel"] > df.loc[i, "Material=Titanium"]:
            mats[i]="Steel"
        else:
            mats[i]="Titanium"
    else:
        if df.loc[i, "Material=Aluminum"] > df.loc[i, "Material=Titanium"]:
            mats[i]="Aluminum"
        else:
            mats[i]="Titanium"
df.drop(["Material=Steel", "Material=Aluminum", "Material=Titanium"], axis=1, inplace=True)
df["Material"]=mats

# #The following only if working in augmented space
# import sys
# sys.path.append('../structural')
# import DataAugment
# df=DataAugment.revfeatureAugment(df)

df.to_csv("Generated Data/DTAI+NSGA.csv")
print(df)

     SSB_Include  CSB_Include  CS Length   BB Drop     Stack      SS E  \
0      -0.471832    -0.163300   0.326457 -0.038168  0.551427  0.061249   
1       1.401671     1.433730   0.409969  0.075565  0.560436  0.048387   
2      -0.348157    -0.086062   0.375138  0.022525  0.565273  0.050291   
3       2.161993     2.458349   0.396687  0.081174  0.557715  0.056971   
4       1.839232     2.326957   0.392484  0.060131  0.555588  0.055641   
..           ...          ...        ...       ...       ...       ...   
245     0.237231     0.300116   0.409568  0.048512  0.581814  0.049772   
246    -0.224438     0.060882   0.361157  0.007050  0.559910  0.076910   
247     0.323694     0.365622   0.411268  0.051112  0.579560  0.048697   
248    -0.092066     0.057972   0.373435  0.002119  0.555683  0.061092   
249    -0.152906     0.024856   0.363131 -0.003141  0.555435  0.063481   

      ST Angle     BB OD     TT OD     HT OD  ...      SS Z  SS Thickness  \
0    69.766738  0.039438  0.048914

In [673]:
# z = tf.random.normal((250, noise_dim), stddev = 0.5, dtype=tf.float32)
# x_fake = model.generator(z, training=False)
HV_ref=evaluation.percentileSelect(y, 0.99)

scaler = StandardScaler()
scaler.fit(y_orig)
y_scaled = scaler.transform(y)
# x_scaled=x_scaled.numpy()
print("real")
print(np.mean(evaluation_function(x_scaled, 0, False)))
print(np.mean(evaluation_function(x_scaled, 0, True)))
print(np.mean(evaluation_function(x_scaled, 1, False)))
print(np.mean(evaluation_function(x_scaled, 1, True)))
print(evaluation.diversity2(x_scaled[:250])[1])
print(evaluation.diversity2(y_scaled[:250])[1])
print(evaluation.HV(y.loc[:250], HV_ref, False))

print("fake")
print(np.mean(evaluation_function(x_fake, 0, False)))
print(np.mean(evaluation_function(x_fake, 0, True)))
print(np.mean(evaluation_function(x_fake, 1, False)))
print(np.mean(evaluation_function(x_fake, 1, True)))
print(np.mean(evaluation_function(x_fake, 1, False, 1)))
print(np.mean(evaluation_function(x_fake, 1, True, 1)))
print(evaluation.diversity2(x_fake.numpy())[1])
inv, _=getInvalid.getInvalid(df)
print((250-len(inv))/250)
y_fake = regressor(x_fake.numpy()).numpy()
y_fake = tf.math.maximum(y_fake, 1e-7).numpy()
y_fake = pd.DataFrame(y_fake, columns=y_orig.columns[:250], index=y_orig.index[:250])
y_fake_scaled = scaler.transform(y_fake)
print(evaluation.diversity2(y_fake_scaled)[1])
print(evaluation.HV(y_fake[:250], HV_ref, False))

real
0.48761854
0.44941565
0.4751873
0.43823296
7.648317914001465
4.388693335257633
(None, 4.159979112032871e-08)
fake
0.7062089
0.6565851
0.741117
0.6850176
0.64483225
0.6434685
7.177391757673741
0.796
0.5426623391856699
(None, 7.908957146084578e-08)
